In [1]:
import pandas as pd
import numpy as np
import re
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, LatentDirichletAllocation, PCA
from sklearn.preprocessing import normalize
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import NMF, LatentDirichletAllocation, PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as accs
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.tree import DecisionTreeClassifier,export_graphviz
import matplotlib.pyplot as plt
import tensorflow as tf
from google.colab import files

Setting up GPU

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
files_upload=files.upload()

Saving 20190928-items.csv to 20190928-items.csv
Saving 20190928-reviews.csv to 20190928-reviews.csv


In [0]:
import io
df2 = pd.read_csv(io.BytesIO(files_upload['20190928-reviews.csv']))

In [0]:
df2.verified=df2.verified.astype('int')

In [6]:
df2.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",0,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",0,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",0,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",0,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",0,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


Finding null values

In [7]:
df2.isnull().sum()

asin                0
name                5
rating              0
date                0
verified            0
title              10
body               20
helpfulVotes    49681
dtype: int64

In [8]:
df2.dtypes

asin             object
name             object
rating            int64
date             object
verified          int64
title            object
body             object
helpfulVotes    float64
dtype: object

In [0]:
df2 = df2[df2['title'].notnull()]
df2 = df2[df2['body'].notnull()]

Filling null values

In [0]:
df2['helpfulVotes'] = df2['helpfulVotes'].fillna(0)


In [0]:
df2.helpfulVotes = df2.helpfulVotes.astype(int)

In [12]:
df2.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",0,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",0,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17
2,B0000SX2UC,Brooke,5,"December 30, 2003",0,Love This Phone,"This is a great, reliable phone. I also purcha...",5
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",0,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",0,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1


In [0]:
def top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

LSA method- Topic Modelling

In [0]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(df2['body'])
nmf = NMF(n_components=30, random_state=1,
          alpha=.1, l1_ratio=.5)
nmf.fit(data_tf_idf)
nmf_data=nmf.transform(data_tf_idf)

In [15]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: camera samsung phones really iphone better time android ve quality
Topic #1: good quality really value deal pretty camera overall looks cellphone
Topic #2: great deal value camera phone quality looks shape features worked
Topic #3: love absolutely new thank note features size big phone galaxy
Topic #4: like new brand looks looked refurbished scratches really box look
Topic #5: excellent quality camera condition value service thanks recommend smartphone performance
Topic #6: phone cell received recommend amazing problems money got refurbished purchased
Topic #7: works perfectly fine looks issues problems time scratches mobile arrived
Topic #8: product described seller service recommend satisfied received time arrived thanks
Topic #9: nice really camera features phone big thanks smartphone size looking
Topic #10: perfect size thanks thank condition gift absolutely arrived exactly looks
Topic #11: el la muy que en es lo teléfono se buen
Topic #12: battery life charge long day ho

PLSA method- Topic Modelling

In [0]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(df2['body'])
nmf = NMF(n_components=30, random_state=1, beta_loss='kullback-leibler',
          solver='mu', alpha=.1, l1_ratio=.5)
nmf_data=nmf.fit(data_tf_idf)

In [17]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: phone like great don getting going thing lot make really
Topic #1: good pretty overall value choice really performance complain stuff real
Topic #2: great works deal value shape thanks freezes freezing friend friendly
Topic #3: love great absolutely color thank learning zero function friendly friends
Topic #4: new like works brand looks perfectly refurbished looked scratches fine
Topic #5: product excellent seller recommend service 100 highly satisfied customer recommended
Topic #6: phone cell perfectly cricket stolen shape metro pcs activated tmobile
Topic #7: battery camera screen day apps life android 10 using works
Topic #8: received return phone disappointed seller said service amazon tried sent
Topic #9: nice big really little looking clean wonderful cheap design durable
Topic #10: perfect condition thank arrived size absolutely needs near early fits
Topic #11: el muy la en lo es que buen bueno se
Topic #12: battery phone life charge day long hours days charged lasts
To

LDA method- Topic Modelling

In [0]:
count = CountVectorizer(stop_words='english', max_features=1000)
data_count = count.fit_transform(df2['body'])
lda = LatentDirichletAllocation(n_components=25, max_iter=5, 
                                learning_method='online', random_state=1)
lda_data=lda.fit_transform(data_count)

In [19]:
top_words(lda, count.get_feature_names(), 10)

Topic #0: new phone came like charger brand box refurbished looks scratches
Topic #1: great works phone excellent product recommend easy use highly s10
Topic #2: love phone price great awesome money wasn s5 waste delivery
Topic #3: phone issues warranty weeks couldn worked tried talk time straight
Topic #4: camera fast phone amazing really price best better quality super
Topic #5: moto nice el la muy que en lo se described
Topic #6: sim card unlocked phone mobile sprint locked work sd unlock
Topic #7: screen pixel use fingerprint photos usb case protector mode display
Topic #8: phone ve phones buy don nokia just time want sony
Topic #9: far happy phone problems bought started s7 old cell s9
Topic #10: perfect condition purchase phone received works quick completely amazon price
Topic #11: android apps google device app version apple update performance software
Topic #12: phone verizon service network customer store took absolutely data motorola
Topic #13: expected ok quality item littl

Analysis on 'title' column

TFID Vectorizer

In [0]:
tf=TfidfVectorizer()
t=tf.fit_transform(df2['title'])

In [0]:
x_train,x_test = train_test_split(t,train_size=0.8,random_state=2)
y_train,y_test = train_test_split(df2['rating'],train_size=0.8,random_state=2)

Random Forest


In [22]:

rf=RandomForestClassifier()
rf.fit(x_train,y_train)
accs(rf.predict(x_test),y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7213263272331945

Multinomial NB

In [23]:
nb=MultinomialNB()
nb.fit(x_train,y_train)
accs(nb.predict(x_test),y_test)

0.7125083046445612

KNN Classifier

In [24]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train)
accs(knn.predict(x_test),y_test)

0.6742163435404964

Decision Tree

In [25]:
tree=DecisionTreeClassifier()
tree.fit(x_train,y_train)
accs(tree.predict(x_test),y_test)

0.6958990155221356

Analysis on the 'body' Column

Tfid Vectorizer

In [0]:
tf=TfidfVectorizer()
t2=tf.fit_transform(df2['body'])

In [0]:
x_train,x_test=train_test_split(t2,train_size=0.8,random_state=1)
y_train,y_test=train_test_split(df2['rating'],train_size=0.8,random_state=1)


Mutinomial NB

In [28]:
nb=MultinomialNB()
nb.fit(x_train,y_train)
accs(nb.predict(x_test),y_test)

0.6588754001328743

Random Forest

In [29]:
rf=RandomForestClassifier()
rf.fit(x_train,y_train)
accs(rf.predict(x_test),y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6824907893942139

KNN

In [30]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)
accs(knn.predict(x_test),y_test)

0.5532403213142477

In [0]:
df2=df2.drop('asin',axis=1)
df2=df2.drop('date',axis=1)
df2=df2.drop('name',axis=1)

Decison Tree Classifier

In [32]:
tree=DecisionTreeClassifier(max_depth=3)
tree.fit(x_train,y_train)
accs(tree.predict(x_test),y_test)

0.5862173098991363

In [0]:
tf=TfidfVectorizer()

In [0]:
df2=df2.dropna(axis=0,subset=['title'])
df2=df2.dropna(axis=0,subset=['body'])

NMF

In [0]:
nmf=NMF(n_components=10)

In [0]:
t_decomposed=nmf.fit_transform(t)

In [0]:
b_decomposed=nmf.fit_transform(t2)

In [38]:
df2.head()

,rating,verified,title,body,helpfulVotes
0,3,0,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1
1,1,0,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17
2,5,0,Love This Phone,"This is a great, reliable phone. I also purcha...",5
3,3,0,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1
4,4,0,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1


In [0]:
df2=df2.drop('title',axis=1)
df2=df2.drop('body',axis=1)

In [40]:
df2.isnull().sum()

rating          0
verified        0
helpfulVotes    0
dtype: int64

In [41]:
df2.head()

,rating,verified,helpfulVotes
0,3,0,1
1,1,0,17
2,5,0,5
3,3,0,1
4,4,0,1


In [0]:
t_decomposed_df=pd.DataFrame(t_decomposed)
b_decomposed_df=pd.DataFrame(b_decomposed)

In [43]:
t_decomposed_df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

In [44]:
b_decomposed_df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

In [0]:
for i in range(b_decomposed_df.shape[1]):
    t_decomposed_df[i+10]=b_decomposed_df[i]
t_decomposed_df[20]=df2['verified']
t_decomposed_df[21]=df2['helpfulVotes']

In [46]:
t_decomposed_df[20] = t_decomposed_df[20].fillna(0)
t_decomposed_df[21] = t_decomposed_df[21].fillna(0)
t_decomposed_df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
dtype: int64

In [0]:
X_train,X_test,Y_train,Y_test= train_test_split(t_decomposed_df,df2.rating, random_state=1,test_size=0.2)

Multinomial NB

In [0]:
nb=MultinomialNB()

In [49]:
X_train.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
dtype: int64

In [50]:
nb.fit(X_train,Y_train)
accs(nb.predict(X_test),Y_test)

0.5315576493326085

Random Forest

In [51]:
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
accs(rf.predict(X_test),Y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.695657425862173

KNN

In [52]:
knn=KNeighborsClassifier()
knn.fit(X_train,Y_train)
accs(knn.predict(X_test),Y_test)

0.6130941595699704

Decision Tree

In [53]:
tree=DecisionTreeClassifier()
tree.fit(X_train,Y_train)
accs(tree.predict(X_test),Y_test)

0.6120674035151296

Nueral Networks

In [0]:
nue_df = pd.read_csv(io.BytesIO(files_upload['20190928-reviews.csv']))

In [55]:
nue_df.body

0        I had the Samsung A600 for awhile which is abs...
1        Due to a software issue between Nokia and Spri...
2        This is a great, reliable phone. I also purcha...
3        I love the phone and all, because I really did...
4        The phone has been great for every purpose it ...
                               ...                        
82810                            Best phone at this price.
82811    If you intend to use this phone on T Mobile be...
82812    Here is my Moto G7 Play complaint: It freezes ...
82813    As far as function works great camera no go wo...
82814    What a great phone! Sleek, fast, great soundin...
Name: body, Length: 82815, dtype: object

In [56]:
from matplotlib import pyplot
from matplotlib import pylab
import sklearn.datasets
import numpy as np
import keras
import keras.datasets
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding, LSTM, GRU, Input
from keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from keras import backend as K
from keras.preprocessing.text import Tokenizer, hashing_trick
from keras.preprocessing.sequence import pad_sequences
import re

Using TensorFlow backend.


In [57]:
train,test= train_test_split(nue_df,train_size=0.8,random_state=1)
train.body

52090    So I bought this phone and took it to a local ...
77124    It has an extremely long battery life, I still...
34046    I sent this item to the Caribbean and the repo...
5292                                           GOOD I LIKE
14288    This is a great phone. Others have mentioned i...
                               ...                        
21440    this Phone had a big scratch on the top..and t...
73349    No wonder this is known to be the king of smar...
50057    I was skeptical ordering a refurbished phone a...
5192           Came as described! Saved $70 not buying new
77708                                                 Good
Name: body, Length: 66252, dtype: object

In [58]:
tokenizer=Tokenizer(num_words=2000)
train.dtypes

asin             object
name             object
rating            int64
date             object
verified           bool
title            object
body             object
helpfulVotes    float64
dtype: object

In [59]:
train.body=train.body.str.replace(r'\d+', '')
test.body=test.body.str.replace(r'\d+', '')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [60]:
train.body=train.body.str.lower()
test.body=test.body.str.lower()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
train=train[train['body'].apply(lambda x: isinstance(x, str))]
test=test[test['body'].apply(lambda x: isinstance(x, str))]

In [62]:
train.body

52090    so i bought this phone and took it to a local ...
77124    it has an extremely long battery life, i still...
34046    i sent this item to the caribbean and the repo...
5292                                           good i like
14288    this is a great phone. others have mentioned i...
                               ...                        
21440    this phone had a big scratch on the top..and t...
73349    no wonder this is known to be the king of smar...
50057    i was skeptical ordering a refurbished phone a...
5192             came as described! saved $ not buying new
77708                                                 good
Name: body, Length: 66235, dtype: object

In [0]:
tokenizer.fit_on_texts(train.body)

In [64]:
train.rating.value_counts()

5    35964
1    13145
4     8572
3     4638
2     3916
Name: rating, dtype: int64

In [0]:
train_Y=keras.utils.to_categorical(train.rating)
test_Y=keras.utils.to_categorical(test.rating)


In [0]:
train_X = pad_sequences(tokenizer.texts_to_sequences(train.body.values), 33)
test_X = pad_sequences(tokenizer.texts_to_sequences(test.body.values),33)

In [67]:
embed_dim = 128
lstm_out = 196
max_features = 2000
model = Sequential()
model.add(Embedding(tokenizer.num_words, tokenizer.num_words, input_length=33))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(.5))
model.add(Conv1D(33, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [68]:
batch_size = 32
model.fit( train_X, 
           train_Y,
           epochs = 100, 
           batch_size=batch_size, 
           verbose = 1, 
           validation_data=(test_X, test_Y))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 66235 samples, validate on 16560 samples
Epoch 1/100





66235/66235 [==============================] - 21s 320us/step - loss: 1.0266 - acc: 0.6427 - val_loss: 0.9143 - val_acc: 0.6818
Epoch 2/100
66235/66235 [==============================] - 15s 229us/step - loss: 0.9204 - acc: 0.6747 - val_loss: 0.8964 - val_acc: 0.6841
Epoch 3/100
66235/66235 [==============================] - 15s 229us/step - loss: 0.8899 - acc: 0.6835 - val_loss: 0.8851 - val_acc: 0.6877
Epoch 4/100
66235/66235 [==============================] - 15s 228us/step - loss: 0.8699 - acc: 0.6896 - val_loss: 0.8782 - val_acc: 0.6856
Epoch 5/100
66235/66235 [==============================] - 15s 232us/step - loss: 0.8505 - acc: 0.6936 - val_loss: 0.8805 - val_acc: 0.6885
Epoch 6/100
66235/66235 [==============================] - 15s 230us/step - loss: 0.8369 - acc: 0.6988 - val_loss: 0.8767 - val_acc: 0.6878
Epoch 7/

In [69]:
nue_df.title

0                              Def not best, but not worst
1                              Text Messaging Doesn't Work
2                                          Love This Phone
3                                  Love the Phone, BUT...!
4             Great phone service and options, lousy case!
                               ...                        
82810    International version. Will require USA adapte...
82811    Be careful of International Version if Using T...
82812            Moto G7 Play Freezes Every Time I Use It.
82813                                   Camera is not good
82814                                             Awesome.
Name: title, Length: 82815, dtype: object

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test = train_test_split(nue_df['title'],train_size=0.8,random_state=1)
y_train,y_test = train_test_split(nue_df['rating'],train_size=0.8,random_state=1)

In [0]:
tokenizer = Tokenizer(num_words=2000)


train.title=train.title.str.replace(r'\d+', '')
test.title=test.title.str.replace(r'\d+', '')

train.title=train.title.str.lower()
test.title=test.title.str.lower()

train=train[train['title'].apply(lambda x: isinstance(x, str))]
test=test[test['title'].apply(lambda x: isinstance(x, str))]

tokenizer.fit_on_texts(train.title)

train_Y=keras.utils.to_categorical(train.rating)
test_Y=keras.utils.to_categorical(test.rating)
#tokenizer.fit_on_texts(x_train)

train_X = pad_sequences(tokenizer.texts_to_sequences(train.title.values), 33)
test_X = pad_sequences(tokenizer.texts_to_sequences(test.title.values),33)



In [0]:
embed_dim = 128
lstm_out = 196
max_features = 2000
model = Sequential()
model.add(Embedding(tokenizer.num_words, tokenizer.num_words, input_length=33))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(Conv1D(66, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(.5))
model.add(Conv1D(33, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [73]:
batch_size = 32
model.fit( train_X, 
           train_Y,
           epochs = 100, 
           batch_size=batch_size, 
           verbose = 1, 
           validation_data=(test_X, test_Y))

Train on 66229 samples, validate on 16556 samples
Epoch 1/100
66229/66229 [==============================] - 16s 234us/step - loss: 1.0182 - acc: 0.6463 - val_loss: 0.8615 - val_acc: 0.6996
Epoch 2/100
66229/66229 [==============================] - 15s 226us/step - loss: 0.8942 - acc: 0.6919 - val_loss: 0.8521 - val_acc: 0.7032
Epoch 3/100
66229/66229 [==============================] - 15s 230us/step - loss: 0.8677 - acc: 0.6990 - val_loss: 0.8476 - val_acc: 0.7041
Epoch 4/100
66229/66229 [==============================] - 15s 226us/step - loss: 0.8503 - acc: 0.7054 - val_loss: 0.8400 - val_acc: 0.7052
Epoch 5/100
66229/66229 [==============================] - 15s 227us/step - loss: 0.8373 - acc: 0.7099 - val_loss: 0.8446 - val_acc: 0.7071
Epoch 6/100
66229/66229 [==============================] - 15s 225us/step - loss: 0.8256 - acc: 0.7129 - val_loss: 0.8387 - val_acc: 0.7054
Epoch 7/100
66229/66229 [==============================] - 15s 229us/step - loss: 0.8156 - acc: 0.7156 - val_l